In [1]:
%%time

from datetime import datetime
import pandas as pd

# from pandasql import sqldf
# pysqldf = lambda q: sqldf(q, globals())
# pysqldf('select * from bounty limit 10')

party = (
  pd.read_parquet('2201-DSA-906-party.parquet')
  ['stash_user_id claim amount'.split()]
  .dropna()
  .rename(columns={'claim':'redeemed_at'})
  .query('"2021-1-1" < redeemed_at < "2022-1-1"')
  .groupby('stash_user_id')
  .sum()
  .reset_index()
)

users = pd.read_parquet('2201-DSA-906-users.parquet')

stockback = (
  pd.read_parquet('2201-DSA-906-stockback.parquet')
  .dropna()
  .query('"2021-1-1" < redeemed_at < "2022-1-1"')
  .groupby('stash_user_uuid')
  .sum()
  .reset_index()
  .merge(users, how='left', on='stash_user_uuid')
  [['stash_user_id','amount']]
)

bounty = (
  pd.read_parquet('2201-DSA-906-bounty.parquet')
  .dropna()
  .rename(columns={'claim':'redeemed_at'})
  .query('"2021-1-1" < redeemed_at < "2022-1-1"')
  .groupby('stash_user_id')
  .sum()
  .reset_index()
  .astype({'stash_user_id': 'int64'})
)

CPU times: user 2min 11s, sys: 21.5 s, total: 2min 32s
Wall time: 2min 29s


In [3]:
%%time

all_promo = (
  pd.concat([bounty,stockback,party])
  .groupby('stash_user_id')
  .sum()
  .reset_index()
  .dropna()
  .query('amount >= 600')
)

all_promo

CPU times: user 14min 16s, sys: 18.9 s, total: 14min 35s
Wall time: 14min 43s


stash_user_id   amount
87332         2064835  2400.56
246629        6152139  30115.0
259485        6471204   2020.0
368952        9502946  1164.32

In [ ]:
profile = pd.read_parquet('2201-DSA-906-profile.parquet')